<a href="https://colab.research.google.com/github/svetaepc/ds_py/blob/master/NB_on_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CountVectorizer
 

https://scikit-learn.ru/6-2-feature-extraction/

CountVectorizer реализует как токенизацию, так и подсчет вхождений в одном классе

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
 
corpus = [
     'This is the first document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?',]

Давайте использовать его для токенизации и подсчета вхождений слов в минималистичном корпусе текстовых документов.  
Каждому члену, найденному анализатором во время подбора, присваивается уникальный целочисленный индекс, соответствующий столбцу в результирующей матрице.


In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(X.toarray())

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]


Конфигурация по умолчанию токенизирует строку, извлекая слова не менее чем из 2 букв. Конкретную функцию, выполняющую этот шаг, можно запросить явно:

In [ ]:
analyze = vectorizer.build_analyzer()
analyze("This is a text document to analyze.") == (
     ['this', 'is', 'text', 'document', 'to', 'analyze'])

True

Обратите внимание, что в предыдущем корпусе первый и последний документы содержат одни и те же слова, поэтому они закодированы в равных векторах. В частности, мы теряем информацию о том, что последний документ является вопросительной формой. Чтобы сохранить некоторую локальную информацию о порядке, мы можем извлечь биграмма или триграммы слов в дополнение к 1 грамму (отдельные слова):

In [ ]:
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(1, 3))
X2 = vectorizer2.fit_transform(corpus)
print(vectorizer2.get_feature_names_out())
print(X2.toarray())

['and' 'and this' 'and this is' 'document' 'document is' 'document is the'
 'first' 'first document' 'is' 'is the' 'is the first' 'is the second'
 'is the third' 'is this' 'is this the' 'one' 'second' 'second document'
 'the' 'the first' 'the first document' 'the second' 'the second document'
 'the third' 'the third one' 'third' 'third one' 'this' 'this document'
 'this document is' 'this is' 'this is the' 'this the' 'this the first']
[[0 0 0 1 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 1 1 0 0]
 [0 0 0 2 1 1 0 0 1 1 0 1 0 0 0 0 1 1 1 0 0 1 1 0 0 0 0 1 1 1 0 0 0 0]
 [1 1 1 0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 1 0 0 0 0 1 1 1 1 1 0 0 1 1 0 0]
 [0 0 0 1 0 0 1 1 1 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 1]]


## TFIDF

В большом текстовом корпусе некоторые слова будут присутствовать очень часто (например, «the», «a», «is» на английском языке), следовательно, несут очень мало значимой информации о фактическом содержании документа. Если бы мы передавали данные прямого подсчета непосредственно классификатору, эти очень частые термины затеняли бы частоты более редких, но более интересных терминов.

Чтобы повторно взвесить функции счетчика в значения с плавающей запятой, подходящие для использования классификатором, очень часто используется преобразование tf – idf.

Tf означает частоту термина, а tf – idf означает частоту термина, умноженную на обратную частоту документа:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(X)

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
  (0, 1)	0.46979138557992045
  (0, 2)	0.5802858236844359
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 8)	0.38408524091481483
  (1, 5)	0.5386476208856763
  (1, 1)	0.6876235979836938
  (1, 6)	0.281088674033753
  (1, 3)	0.281088674033753
  (1, 8)	0.281088674033753
  (2, 4)	0.511848512707169
  (2, 7)	0.511848512707169
  (2, 0)	0.511848512707169
  (2, 6)	0.267103787642168
  (2, 3)	0.267103787642168
  (2, 8)	0.267103787642168
  (3, 1)	0.46979138557992045
  (3, 2)	0.5802858236844359
  (3, 6)	0.38408524091481483
  (3, 3)	0.38408524091481483
  (3, 8)	0.38408524091481483


##word2vec

В реальных приложениях модели Word2Vec создаются с использованием миллиардов документов. Например, модель Google Word2Vec обучается с использованием 3 миллионов слов и фраз.


In [ ]:
import bs4 as bs 
import urllib.request 
import re 
import nltk 
scrapped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Artificial_intelligence') 
article = scrapped_data .read() 
parsed_article = bs.BeautifulSoup(article,'lxml') 
paragraphs = parsed_article.find_all('p') 
article_text = "" 
for p in paragraphs: 
  article_text += p.text


Во-первых, нам нужно преобразовать нашу статью в предложения. Мы используем утилиту nltk.sent_tokenize для преобразования нашей статьи в предложения. Для преобразования предложений в слова мы используем утилиту nltk.word_tokenize. На последнем этапе предварительной обработки мы удаляем из текста все стоп-слова. После того, как скрипт завершит свое выполнение, объект all_words содержит список всех слов в статье. Мы будем использовать этот список для создания нашей модели Word2Vec с библиотекой Gensim.

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
# Cleaing the text 
processed_article = article_text.lower() 
processed_article = re.sub('[^a-zA-Z]', ' ', processed_article ) 
processed_article = re.sub(r'\s+', ' ', processed_article) 
# Preparing the dataset 
all_sentences = nltk.sent_tokenize(processed_article) 
all_words = [nltk.word_tokenize(sent) for sent in all_sentences] 
# Removing Stop Words 
from nltk.corpus import stopwords 
for i in range(len(all_words)): 
  all_words[i] = [w for w in all_words[i] 
                  if w not in stopwords.words('english')]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Значение 2 для min_count указывает на включение в модель Word2Vec только тех слов, которые встречаются в корпусе как минимум дважды.

In [ ]:
from gensim.models import Word2Vec 
word2vec = Word2Vec(all_words, min_count=2)


In [ ]:
vocabulary = word2vec.wv.key_to_index 
print(vocabulary)


{'ai': 0, 'intelligence': 1, 'artificial': 2, 'learning': 3, 'machine': 4, 'human': 5, 'problems': 6, 'networks': 7, 'many': 8, 'research': 9, 'used': 10, 'also': 11, 'knowledge': 12, 'search': 13, 'neural': 14, 'use': 15, 'symbolic': 16, 'researchers': 17, 'may': 18, 'logic': 19, 'general': 20, 'field': 21, 'systems': 22, 'machines': 23, 'computer': 24, 'problem': 25, 'would': 26, 'humans': 27, 'data': 28, 'reasoning': 29, 'algorithms': 30, 'mind': 31, 'applications': 32, 'solve': 33, 'tools': 34, 'intelligent': 35, 'goals': 36, 'could': 37, 'computing': 38, 'one': 39, 'specific': 40, 'system': 41, 'include': 42, 'example': 43, 'decision': 44, 'since': 45, 'developed': 46, 'two': 47, 'number': 48, 'optimization': 49, 'approaches': 50, 'ability': 51, 'program': 52, 'mathematical': 53, 'recognition': 54, 'risk': 55, 'however': 56, 'based': 57, 'world': 58, 'information': 59, 'including': 60, 'theory': 61, 'u': 62, 'difficult': 63, 'well': 64, 'others': 65, 'using': 66, 'deep': 67, 'firs

In [ ]:
v1 = word2vec.wv['artificial']
v1

array([-1.60073338e-03,  5.08260261e-03, -5.05369715e-03, -1.19461026e-03,
        7.02408003e-03,  1.75912608e-03, -2.59982119e-03,  1.06169907e-02,
       -1.11749945e-02,  4.65675117e-03, -7.50510907e-03, -7.01052137e-03,
        8.05803854e-03,  2.96750921e-03,  7.74817262e-03, -7.71344220e-03,
        6.00213325e-03,  6.53982535e-03, -9.62967146e-03, -1.19432686e-02,
       -5.28674945e-03, -3.40612722e-03, -3.16013675e-03, -1.07988175e-02,
        6.45832065e-03, -3.98814818e-03,  4.86717559e-03,  2.31748703e-03,
       -8.79074819e-03,  4.68504196e-03,  1.07759265e-02, -5.18562971e-03,
       -5.22886589e-03, -4.26600734e-03, -1.11535434e-02,  2.53374968e-03,
       -1.65568665e-04, -7.47654471e-04, -1.11759116e-03, -8.52265581e-03,
       -6.12793630e-03, -9.06595727e-04, -1.25828164e-03,  5.86924329e-03,
        5.89350192e-03,  1.82834442e-03, -5.02599054e-04, -2.67082779e-03,
       -1.57851016e-03,  7.01579324e-04,  3.39879980e-03, -6.07468095e-03,
       -8.02578218e-03, -

In [ ]:
sim_words = word2vec.wv.most_similar('intelligence')
sim_words

[('smart', 0.4090029001235962),
 ('may', 0.3627232015132904),
 ('humans', 0.3279481530189514),
 ('ai', 0.3214792013168335),
 ('systems', 0.31423211097717285),
 ('methods', 0.3131225109100342),
 ('ability', 0.30731719732284546),
 ('machines', 0.30406704545021057),
 ('search', 0.2984282076358795),
 ('economics', 0.2977701425552368)]

## Распознание спама


In [ ]:
#Загружаем данные и разархивируем

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip

--2023-04-17 10:10:02--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z 100%[===================>] 198.65K   734KB/s    in 0.3s    

2023-04-17 10:10:03 (734 KB/s) - ‘smsspamcollection.zip’ saved [203415/203415]



In [ ]:
!unzip smsspamcollection.zip

Archive:  smsspamcollection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


Пакет torch содержит структуры данных для многомерных тензоров и определяет математические операции над этими тензорами. Кроме того, он предоставляет множество утилит для эффективной сериализации тензоров и произвольных типов, а также другие полезные утилиты.

In [ ]:
import torch
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_table('SMSSpamCollection',sep='\t',header=None, names=['label','sms_message'])

In [ ]:
#Данные 

In [ ]:
df.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### Наивный Байесовский классификатор

In [ ]:
#заменяем целевую переменную

In [ ]:
df['label'] = df.label.map({'ham':0,'spam':1})
df.head()

,label,sms_message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
#Делим на трейновую и тестовую выборку
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], df['label'], random_state=1)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#Считаем количество слов в предложениях
#получим матрицу размером (количество предложений Х размер словаря)

count_vector = CountVectorizer()
training_data = count_vector.fit_transform(X_train).toarray().astype('float')
testing_data = count_vector.transform(X_test).toarray().astype('float')

# переводим в тензоры
train_tensor = torch.tensor(training_data)
test_tensor = torch.tensor(testing_data)

In [ ]:
# размер трейновой выборки
training_data.shape

(4179, 7456)

In [ ]:
# всего слов в предложениях
train_tensor.sum()

tensor(59839., dtype=torch.float64)

In [ ]:
# тензор с предложениями класса spam
spam_train_tensor = train_tensor[(y_train == 1).values]

# тензор с предложениями класса not_spam
not_spam_train_tensor = train_tensor[(y_train == 0).values]
spam_train_tensor.shape, not_spam_train_tensor.shape

(torch.Size([562, 7456]), torch.Size([3617, 7456]))

In [ ]:

#Вероятность слова при условии что предложение спам
p_w_spam = (spam_train_tensor.sum(axis=0)) / (spam_train_tensor.sum())

#Вероятность слова при условии что предложение не спам
p_w_not_spam = (not_spam_train_tensor.sum(axis=0)) / (not_spam_train_tensor.sum())

p_w_spam, p_w_not_spam


(tensor([0.0004, 0.0018, 0.0002,  ..., 0.0000, 0.0000, 0.0000],
        dtype=torch.float64),
 tensor([0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 2.1413e-05, 2.1413e-05,
         2.1413e-05], dtype=torch.float64))

In [ ]:
# вероятность, что любое сообщение спам
p_spam = (y_train == 1).values.sum() / len(y_train)

# вероятность, что любое сообщение не спам
p_not_spam = (y_train == 0).values.sum() / len(y_train)

In [ ]:
#проверим на одном семпле
test_sample = test_tensor[0]

In [ ]:
# посчитает значение за спам 
# пляски с прибавлением числа e и вычитания единицы используются для того,
# что бы небыло минус бесконечности после взятия логарифма и суммировались неотрицательные значения
np.log(p_spam+2.71828182846)-1 + (((test_sample*p_w_spam)+2.71828182846).log()-1).sum()

tensor(0.0535, dtype=torch.float64)

In [ ]:
#посчитаем значение против спама
np.log(p_not_spam+2.71828182846)-1 + (((test_sample*p_w_not_spam)+2.71828182846).log()-1).sum()

tensor(0.2841, dtype=torch.float64)

In [ ]:
# видно что значение против спама больше, чем значение за спам 
# соответственно значение не спам

In [ ]:
#посчитаем для всей тестовой выборки

#размер тестовой выборки
test_tensor.shape

torch.Size([1393, 7456])

In [ ]:
#посчитаем предсказания как сравнение величинв за спам и против спама
y_pred = (np.log(p_spam+2.71828182846)-1 + (((test_tensor*p_w_spam)+2.71828182846).log()-1).sum(dim=1)) >= \
 np.log(p_not_spam+2.71828182846)-1 + (((test_tensor*p_w_not_spam)+2.71828182846).log()-1).sum()

In [ ]:
# предсказанные значения
y_pred.int()

tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)

In [ ]:
#истиные значение
y_test.to_numpy()

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
#сравним реальные метки с предсказанными
test = (y_pred == torch.tensor(y_test.to_numpy()))

In [ ]:
#посчитаем точность модели как отношение количество совпадений к размеру выборки  
test.sum().item()/test.shape[0]

0.8671931083991385

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# обучить классификатор
classifier = MultinomialNB()
classifier.fit(train_tensor, y_train)
predicted = classifier.predict(test_tensor) 

In [ ]:
predicted.shape

(1393,)

In [ ]:
y_test.shape

(1393,)

Как мы видим, количество ошибок довольно сбалансировано между законным и спамом: 5 легитимных сообщения классифицируются как спам, а 11 спам-сообщений классифицируются как легитимные. В целом, это очень хорошие результаты для нашего простого классификатора.

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
 
print(confusion_matrix(y_test, predicted)) 

[[1203    5]
 [  11  174]]


In [ ]:
accuracy_score(y_test, predicted)

0.9885139985642498

In [ ]:
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      1208
           1       0.97      0.94      0.96       185

    accuracy                           0.99      1393
   macro avg       0.98      0.97      0.97      1393
weighted avg       0.99      0.99      0.99      1393

